# SAE-78: Document Embeddings Demo

Ce notebook démontre l'utilisation de `compute_doc_embeddings` pour créer des vecteurs de documents (reviews) à partir du modèle Word2Vec.

In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Ajouter le dossier src au path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.features import compute_doc_embeddings
from src.text_preprocessing import preprocess_text, tokenize_text

## 1. Chargement des données

In [ ]:
# Charger un échantillon de reviews
DATA_PATH = '../data/cleaned/reviews_clean.parquet'

try:
    df = pd.read_parquet(DATA_PATH)
    # Échantillonner pour la démo
    df_sample = df.sample(n=1000, random_state=42)
    print(f"Loaded {len(df_sample)} reviews.")
except FileNotFoundError:
    print(f"File not found: {DATA_PATH}")
    DATA_PATH = 'data/cleaned/reviews_clean.parquet'
    df = pd.read_parquet(DATA_PATH)
    df_sample = df.sample(n=1000, random_state=42)
    print(f"Loaded {len(df_sample)} reviews (fallback path).")

## 2. Tokenisation (si nécessaire)

In [ ]:
# On s'assure d'avoir une colonne de tokens
# Idéalement 'text_preprocessed' devrait déjà être tokenisé ou être une chaine propre

if 'text_preprocessed' not in df_sample.columns:
    print("Preprocessing text...")
    df_sample['text_preprocessed'] = df_sample['text'].apply(lambda x: preprocess_text(str(x)))

# Tokenisation simple pour l'embedding (split)
# Si preprocess_text renvoie une str, on split.
df_sample['tokens'] = df_sample['text_preprocessed'].apply(lambda x: x.split() if isinstance(x, str) else x)

print(df_sample[['text_preprocessed', 'tokens']].head())

## 3. Calcul des Embeddings

In [ ]:
MODEL_PATH = '../outputs/models/word2vec_yelp.model'
if not os.path.exists(MODEL_PATH):
     MODEL_PATH = 'outputs/models/word2vec_yelp.model'

embeddings = compute_doc_embeddings(df_sample['tokens'], model_path=MODEL_PATH)

print(f"Embeddings shape: {embeddings.shape}")

## 4. visualisation (Optionnel)

In [ ]:
# Exemple d'un vecteur
print("Premier vecteur (extrait):")
print(embeddings[0][:10])